# Explainable Outcome Prediction - Preprocessing and EDA

- Author: David Steiner
- December 2021

In [ ]:
!pip install imblearn
!pip install pm4py
!pip install seaborn

!pip install interpret
!pip install xgboost
!pip install catboost

!pip install keras
!pip install tensorflow --user
!pip install keras-tuner

In [2]:
from prep_custom import get_dataset_settings, xes_converter, add_timestap_features, add_inter_case_features, impute_missing_values
from prep_custom import cut_trace_before_activity, create_trace_bucket,remove_features,remove_events
from prep_custom import split_data_temporal,replace_missing_cols,prepare_ml_train_test, aggregate_data, group_infrequent_features
from prep_custom import define_binary_outcome_label

import pandas as pd
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 100)

import numpy as np

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

# Data Preprocessing

### Available XES Eventlogs :

- BPI_Challenge_2013_incidents.xes.gz
- BPI Challenge 2017.xes.gz
- BPI_Challenge_2019.xes
- Hospital Billing - Event Log.xes.gz

In [3]:
%%time
data = xes_converter('BPI Challenge 2017.xes.gz')
print('Shape:', data.shape)
data.head(5)

parsing log, completed traces ::   0%|          | 0/31509 [00:00<?, ?it/s]

Shape: (1202267, 19)
Wall time: 2min 24s


,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
case_id_col = "case:concept:name"   # Case ID
activity_col = "concept:name"       # Event Name
timestamp_col = 'time:timestamp'    
label_col = 'label'                 # Output Label

#Display shape and one full case
data[data[case_id_col]==data[case_id_col].unique()[0]]

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,statechange,User_1,A_Concept,Application,ApplState_642383566,complete,2016-01-01 09:52:36.413000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Obtained,User_17,W_Complete application,Workflow,Workitem_1875340971,start,2016-01-02 10:45:22.429000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Released,User_17,W_Complete application,Workflow,Workitem_1452291795,suspend,2016-01-02 10:49:28.816000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,statechange,User_52,A_Accepted,Application,ApplState_99568828,complete,2016-01-02 11:23:04.299000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Created,User_52,O_Create Offer,Offer,Offer_148581083,complete,2016-01-02 11:29:03.994000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,20000.0,44.0,True,498.29,True,979.0,20000.0,NaN


In [5]:
%%time
data = add_timestap_features(data)
data = add_inter_case_features(data)
data = impute_missing_values(data)

#Display shape and one full case
print('Shape:', data.shape)
data[data[case_id_col]==data[case_id_col].unique()[0]]

Extracting timestamp features
Extracting open cases
Imputing missing values
Shape: (1202267, 28)
Wall time: 1h 32min 1s


,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,timesincemidnight,month,weekday,hour,timesincelastevent,timesincecasestart,event_nr,open_cases,label
686058,Created,User_1,A_Create Application,Application,Application_1000086665,complete,2016-08-03 15:57:21.673000+00:00,"Other, see explanation",New credit,Application_1000086665,5000.0,0.0,0.0,missing,0.00,missing,0.0,0.0,missing,957,8,2,15,0.000000,0.000000,1,2250,empty
686059,statechange,User_1,A_Submitted,Application,ApplState_161925113,complete,2016-08-03 15:57:21.734000+00:00,"Other, see explanation",New credit,Application_1000086665,5000.0,0.0,0.0,missing,0.00,missing,0.0,0.0,missing,957,8,2,15,0.001017,0.001017,2,2250,empty
686060,Created,User_1,W_Handle leads,Workflow,Workitem_747707399,schedule,2016-08-03 15:57:21.963000+00:00,"Other, see explanation",New credit,Application_1000086665,5000.0,0.0,0.0,missing,0.00,missing,0.0,0.0,missing,957,8,2,15,0.003817,0.004833,3,2250,empty
686061,Deleted,User_1,W_Handle leads,Workflow,Workitem_1030261128,withdraw,2016-08-03 15:58:28.286000+00:00,"Other, see explanation",New credit,Application_1000086665,5000.0,0.0,0.0,missing,0.00,missing,0.0,0.0,missing,958,8,2,15,1.105383,1.110217,4,2250,empty
686062,Created,User_1,W_Complete application,Workflow,Workitem_1127124826,schedule,2016-08-03 15:58:28.293000+00:00,"Other, see explanation",New credit,Application_1000086665,5000.0,0.0,0.0,missing,0.00,missing,0.0,0.0,missing,958,8,2,15,0.000117,1.110333,5,2250,empty
686063,statechange,User_1,A_Concept,Application,ApplState_385184570,complete,2016-08-03 15:58:28.299000+00:00,"Other, see explanation",New credit,Application_1000086665,5000.0,0.0,0.0,missing,0.00,missing,0.0,0.0,missing,958,8,2,15,0.000100,1.110433,6,2250,empty
686064,Obtained,User_14,W_Complete application,Workflow,Workitem_518019192,start,2016-08-04 13:39:29.557000+00:00,"Other, see explanation",New credit,Application_1000086665,5000.0,0.0,0.0,missing,0.00,missing,0.0,0.0,missing,819,8,3,13,1301.020967,1302.131400,7,2192,empty
686065,Released,User_14,W_Complete application,Workflow,Workitem_1967765821,suspend,2016-08-04 13:50:12.281000+00:00,"Other, see explanation",New credit,Application_1000086665,5000.0,0.0,0.0,missing,0.00,missing,0.0,0.0,missing,830,8,3,13,10.712067,1312.843467,8,2188,empty
686066,statechange,User_5,A_Accepted,Application,ApplState_856156982,complete,2016-08-05 13:57:07.419000+00:00,"Other, see explanation",New credit,Application_1000086665,5000.0,0.0,0.0,missing,0.00,missing,0.0,0.0,missing,837,8,4,13,1446.918967,2759.762433,9,2191,empty
686067,Created,User_5,O_Create Offer,Offer,Offer_410892064,complete,2016-08-05 13:59:57.320000+00:00,"Other, see explanation",New credit,Application_1000086665,5000.0,5000.0,22.0,True,241.28,False,0.0,5000.0,missing,839,8,4,13,2.831683,2762.594117,10,2192,empty


In [6]:
#Export preprocessed dataset
data.to_csv("BPIC_17.csv", index = False)

## EDA 

### Available Datasets:

- BPIC13
- BPIC17
- BPIC19
- BPICHospital

In [8]:
data, case_id_col, activity_col, timestamp_col, label_col, resource_col, event_categorical_attributes, event_numeric_attributes, case_categorical_attributes, case_numeric_attributes, static_cols, dynamic_cols, cat_cols = get_dataset_settings('BPIC17')
attributes = [event_categorical_attributes, case_categorical_attributes, event_numeric_attributes, case_numeric_attributes]

Categoric Event Attributes: 8 ['Action', 'org:resource', 'concept:name', 'EventID', 'lifecycle:transition', 'Accepted', 'Selected', 'OfferID'] 

Numeric Event Attributes: 13 ['FirstWithdrawalAmount', 'NumberOfTerms', 'MonthlyCost', 'CreditScore', 'OfferedAmount', 'timesincemidnight', 'month', 'weekday', 'hour', 'timesincelastevent', 'timesincecasestart', 'event_nr', 'open_cases'] 

Categoric Case Attributes: 3 ['EventOrigin', 'case:LoanGoal', 'case:ApplicationType'] 

Numeric Case Attributes: 1 ['case:RequestedAmount'] 

Dataset Shape (1202267, 28)


In [9]:
data.aggregate(pd.Series.nunique).sort_values(ascending=False)

EventID                  1202267
time:timestamp           1202209
timesincecasestart       1078507
timesincelastevent        446885
OfferID                    42996
case:concept:name          31509
FirstWithdrawalAmount       5930
MonthlyCost                 5817
open_cases                  2707
timesincemidnight           1400
case:RequestedAmount         701
OfferedAmount                664
CreditScore                  520
event_nr                     180
org:resource                 149
NumberOfTerms                148
concept:name                  26
hour                          24
case:LoanGoal                 14
month                         12
lifecycle:transition           7
weekday                        7
Action                         5
Selected                       3
Accepted                       3
EventOrigin                    3
case:ApplicationType           2
label                          1
dtype: int64

In [10]:
print("--- event_numeric_attributes ---")
for col in event_numeric_attributes:
    print(col, '=', len(data[col].unique()), 'unique values', sep=' ')
    print(data[col].describe())
    print()
    
print("--- case_numeric_attributes ---")
for col in case_numeric_attributes:
    print(col, '=', len(data[col].unique()), 'unique values', sep=' ')
    print(data[col].describe())
    print()

--- event_numeric_attributes ---
FirstWithdrawalAmount = 5930 unique values
count    1.202267e+06
mean     6.410647e+03
std      9.987348e+03
min      0.000000e+00
25%      0.000000e+00
50%      1.719000e+03
75%      1.000000e+04
max      7.500000e+04
Name: FirstWithdrawalAmount, dtype: float64

NumberOfTerms = 148 unique values
count    1.202267e+06
mean     6.506787e+01
std      4.804358e+01
min      0.000000e+00
25%      2.300000e+01
50%      6.000000e+01
75%      1.200000e+02
max      1.800000e+02
Name: NumberOfTerms, dtype: float64

MonthlyCost = 5817 unique values
count    1.202267e+06
mean     2.175509e+02
std      2.036598e+02
min      0.000000e+00
25%      7.000000e+01
50%      1.984000e+02
75%      3.000700e+02
max      6.673830e+03
Name: MonthlyCost, dtype: float64

CreditScore = 520 unique values
count    1.202267e+06
mean     3.512002e+02
std      4.408910e+02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      8.640000e+02
max      1.145000e+03
Name

In [11]:
print("--- Dynamic Cat Columns ---")
for col in event_categorical_attributes:
    print(col, '=', len(data[col].unique()), 'unique values', sep=' ')
    print(data[col].unique())
    print()
    
print("--- case_categorical_attributes ---")
for col in case_categorical_attributes:
    print(col, '=', len(data[col].unique()), 'unique values', sep=' ')
    print(data[col].unique())
    print()

--- Dynamic Cat Columns ---
Action = 5 unique values
['Created' 'statechange' 'Deleted' 'Obtained' 'Released']

org:resource = 149 unique values
['User_1' 'User_14' 'User_5' 'User_18' 'User_32' 'User_118' 'User_90'
 'User_71' 'User_16' 'User_132' 'User_91' 'User_119' 'User_58' 'User_121'
 'User_68' 'User_28' 'User_3' 'User_29' 'User_75' 'User_24' 'User_25'
 'User_87' 'User_37' 'User_116' 'User_100' 'User_30' 'User_15' 'User_77'
 'User_53' 'User_133' 'User_67' 'User_123' 'User_7' 'User_23' 'User_126'
 'User_95' 'User_99' 'User_98' 'User_81' 'User_69' 'User_72' 'User_94'
 'User_51' 'User_78' 'User_134' 'User_106' 'User_42' 'User_34' 'User_109'
 'User_48' 'User_120' 'User_102' 'User_43' 'User_115' 'User_13' 'User_27'
 'User_85' 'User_4' 'User_46' 'User_40' 'User_49' 'User_131' 'User_62'
 'User_56' 'User_41' 'User_112' 'User_10' 'User_96' 'User_79' 'User_63'
 'User_80' 'User_83' 'User_52' 'User_60' 'User_39' 'User_122' 'User_2'
 'User_31' 'User_21' 'User_129' 'User_19' 'User_33' 'User_93' 

In [12]:
print(data.groupby(activity_col)[case_id_col].count().sort_values(ascending=False))

concept:name
W_Validate application        209496
W_Call after offers           191092
W_Call incomplete files       168529
W_Complete application        148900
W_Handle leads                 47264
O_Create Offer                 42995
O_Created                      42995
O_Sent (mail and online)       39707
A_Validating                   38816
A_Create Application           31509
A_Concept                      31509
A_Accepted                     31509
A_Complete                     31362
O_Returned                     23305
A_Incomplete                   23055
O_Cancelled                    20898
A_Submitted                    20423
O_Accepted                     17228
A_Pending                      17228
A_Cancelled                    10431
O_Refused                       4695
A_Denied                        3753
W_Assess potential fraud        3282
O_Sent (online only)            2026
W_Shortened completion           238
W_Personal Loan collection        22
Name: case:concept:name, 

In [13]:
data.groupby(case_id_col)[activity_col].count().sort_values()

case:concept:name
Application_1515064385     10
Application_1829767138     10
Application_968189026      10
Application_961957338      10
Application_523628252      10
                         ... 
Application_1387439149    154
Application_1875888758    156
Application_1359726788    159
Application_2037628374    180
Application_1219772874    180
Name: concept:name, Length: 31509, dtype: int64

In [14]:
#Display shape and one full case
data[data[case_id_col]==data[case_id_col].unique()[10]]

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,timesincemidnight,month,weekday,hour,timesincelastevent,timesincecasestart,event_nr,open_cases,label
344,Created,User_1,A_Create Application,Application,Application_1000610355,complete,2016-11-30 08:30:06.392000+00:00,Not speficied,New credit,Application_1000610355,10000.0,0.0,0.0,missing,0.0,missing,0.0,0.0,missing,510,11,2,8,0.000000,0.000000,1,1827,empty
345,statechange,User_1,A_Submitted,Application,ApplState_301801710,complete,2016-11-30 08:30:07.554000+00:00,Not speficied,New credit,Application_1000610355,10000.0,0.0,0.0,missing,0.0,missing,0.0,0.0,missing,510,11,2,8,0.019367,0.019367,2,1827,empty
346,Created,User_1,W_Handle leads,Workflow,Workitem_1519952104,schedule,2016-11-30 08:30:07.724000+00:00,Not speficied,New credit,Application_1000610355,10000.0,0.0,0.0,missing,0.0,missing,0.0,0.0,missing,510,11,2,8,0.002833,0.022200,3,1827,empty
347,Obtained,User_5,W_Handle leads,Workflow,Workitem_36541799,start,2016-11-30 12:00:33.801000+00:00,Not speficied,New credit,Application_1000610355,10000.0,0.0,0.0,missing,0.0,missing,0.0,0.0,missing,720,11,2,12,210.434617,210.456817,4,1805,empty
348,Deleted,User_5,W_Handle leads,Workflow,Workitem_2031265715,complete,2016-11-30 12:00:49.566000+00:00,Not speficied,New credit,Application_1000610355,10000.0,0.0,0.0,missing,0.0,missing,0.0,0.0,missing,720,11,2,12,0.262750,210.719567,5,1805,empty
349,Created,User_5,W_Complete application,Workflow,Workitem_1601853822,schedule,2016-11-30 12:00:49.571000+00:00,Not speficied,New credit,Application_1000610355,10000.0,0.0,0.0,missing,0.0,missing,0.0,0.0,missing,720,11,2,12,0.000083,210.719650,6,1805,empty
350,Obtained,User_5,W_Complete application,Workflow,Workitem_1816633637,start,2016-11-30 12:00:49.572000+00:00,Not speficied,New credit,Application_1000610355,10000.0,0.0,0.0,missing,0.0,missing,0.0,0.0,missing,720,11,2,12,0.000017,210.719667,7,1805,empty
351,statechange,User_5,A_Concept,Application,ApplState_627387221,complete,2016-11-30 12:00:49.574000+00:00,Not speficied,New credit,Application_1000610355,10000.0,0.0,0.0,missing,0.0,missing,0.0,0.0,missing,720,11,2,12,0.000033,210.719700,8,1805,empty
352,Released,User_5,W_Complete application,Workflow,Workitem_965530268,suspend,2016-11-30 12:01:19.622000+00:00,Not speficied,New credit,Application_1000610355,10000.0,0.0,0.0,missing,0.0,missing,0.0,0.0,missing,721,11,2,12,0.500800,211.220500,9,1805,empty
353,Obtained,User_69,W_Complete application,Workflow,Workitem_556328765,resume,2016-12-01 08:51:28.530000+00:00,Not speficied,New credit,Application_1000610355,10000.0,0.0,0.0,missing,0.0,missing,0.0,0.0,missing,531,12,3,8,1250.148467,1461.368967,10,1763,empty


## Dataset Statistics

In [ ]:
def get_dataset_stat(dataset):
    data, case_id_col, activity_col, timestamp_col, label_col, resource_col, event_categorical_attributes, event_numeric_attributes, case_categorical_attributes, case_numeric_attributes, static_cols, dynamic_cols, cat_cols = get_dataset_settings(dataset)
    n_event_categorical_attributes = len(event_categorical_attributes)
    n_event_numeric_attributes = len(event_numeric_attributes) -8
    n_case_categorical_attributes = len(case_categorical_attributes)
    n_case_numeric_attributes = len(case_numeric_attributes)

    n_events = len(data[activity_col].unique())
    tmp = data[[case_id_col, 'event_nr']].groupby([case_id_col]).max()
    n_event_length_max = tmp['event_nr'].max()
    n_event_length_min = tmp['event_nr'].min()
    n_event_length_mean = int(tmp['event_nr'].mean())

    n_cases = len(data[case_id_col].unique())
    
    tmp = data[[case_id_col, 'timesincecasestart']].groupby([case_id_col]).max()
    n_mean_case_duration = round((tmp['timesincecasestart'].mean()/360),1)
    
    dataset_stats = pd.DataFrame()

    dataset_stats = dataset_stats.append(pd.DataFrame({
      'Dataset':[dataset],
      'Cases':[n_cases],
      'Min Case Length': [n_event_length_min], 
      'Max Case Length':[n_event_length_max], 
      'Mean Case Length':[n_event_length_mean], 
      'Mean Duration (hours)':[n_mean_case_duration],
      'Events':[n_events],
      'Cat. Event Attr.':[n_event_categorical_attributes],
      'Num. Event Attr.':[n_event_numeric_attributes],
      'Cat. Case Attr.':[n_case_categorical_attributes],
      'Num. Case Attr. ':[n_case_numeric_attributes]}))
    
    return dataset_stats

In [ ]:
datasets = ['BPIC13', 'BPIC17' ,'BPIC19', 'BPICHospital']

dataset_stats = pd.DataFrame()

for dataset in datasets:
    print(dataset)
    dataset_stat = get_dataset_stat(dataset)
    dataset_stats = dataset_stats.append(dataset_stat)
    
dataset_stats.to_csv("Dataset_Stats.csv", sep=";")
dataset_stats

## Business Outcomes Statistics

In [ ]:
outcome_target_list = [
'BPIC17-LoanAccepted',
'BPIC17-PotentialFraud',
'BPIC17-LongRunningCases',
'BPIC13-SupportLevel-1',
'BPIC19-DeletedPO',
'BPICHospital-BillingClosed',
'BPICHospital-CaseReopened',
]

outcome_stats = pd.DataFrame()

for outcome in outcome_target_list:
    print(outcome)
    eventlog = outcome.split('-')[0]

    data, case_id_col, activity_col, timestamp_col, label_col, resource_col, event_categorical_attributes, event_numeric_attributes, case_categorical_attributes, case_numeric_attributes, static_cols, dynamic_cols, cat_cols = get_dataset_settings(eventlog)
    attributes = [event_categorical_attributes, case_categorical_attributes, event_numeric_attributes, case_numeric_attributes]
    data_labeled, drop_events_list, dl_attributes = define_binary_outcome_label(data, attributes, outcome_label=outcome)

    tmp = data_labeled[[case_id_col, label_col]].groupby(case_id_col).max()
    class_1 = tmp[label_col].sum()
    class_0 = len(tmp) - class_1
    class_ratio = round(class_1  / class_0,4)


    outcome_stats = outcome_stats.append(pd.DataFrame({
      'Outcome':[outcome],
      'Class 1':[class_1],
      'Class 0 ': [class_0], 
      'Class Ratio':[class_ratio]
    }))

        
outcome_stats.to_csv("Outcome_Stats.csv", sep=";")

outcome_stats